In [1]:
import pandas as pd
import numpy as np
import re
import ast

def isbn_check(isbn):
    if type(isbn) == float:
        isbn = '{:.0f}'.format(isbn)
    elif type(isbn) == int:
        isbn = '{:d}'.format(isbn)
        
    # Remove dashes
    isbn = isbn.replace('-','')
    
    if len(isbn) == 10 and re.match(r'^[0-9]*$', isbn[0:9]) is not None:
        isbn = '978' + isbn[0:9] + '0'
        
    if len(isbn) == 13 and re.match(r'^[0-9]*$', isbn) is not None:
        check = 0
        for i, ch in enumerate(isbn[0:12]):
            check += int(ch) * (1 + i % 2 * 2)

        check = check % 10
        if check > 0:
            check = 10 - check

        return isbn[0:12]+str(check)
    else:
        return ''

In [3]:
books_db = pd.DataFrame(columns=('title', 'authors', 'isbn', 'image', 'publisher', 'year', 'genre', 'lang'))

# Read Amadeus price
b = pd.read_csv('gs://biblosphere-210106.appspot.com/datasets/prices/CLEAN-hit_price_nov2020_assort - Лист1.csv', usecols = ['authors', 'title', 'year', 'isbn', 'publisher'], \
               names = ['authors', 'title', 'year', 'isbn', 'publisher'], \
               header=1)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(isbn_check)
# TODO: Process books without ISBN
#noisbn = b[b.isbn == '']
b = b[b.isbn != '']

books_db = books_db.append(b, ignore_index=True, sort=True)




Number of books (raw): 61439


,authors,title,year,isbn,publisher
0,Арефьева О.В.,Гая.Live Book.Одностишия-2.Сборник ...,2016.0,978-5-9907254-5-4,Livebook
1,Барнетт С.,Гая.Live Book.Занимательное дождеведение.Дождь...,2017.0,978-5-9908083-3-1,Livebook
2,Беттс А.Д.,Гая.Live Book.Зак и Мия ...,2015.0,978-5-9905810-8-1,Livebook
3,"Блэквелл Э., Ли А.",Гая.Ты сможешь ...,2020.0,978-5-907056-53-4,Livebook
4,Бонфильоли К.,Гая.Live Book.Бонфильоли-1.Не тычьте в меня эт...,2015.0,978-5-9905810-1-2,Livebook


In [52]:
# Script to extract books from PRICE data

books_db = pd.DataFrame(columns=('title', 'authors', 'isbn', 'image', 'publisher', 'year', 'genre', 'lang'))

# Read 366 price
b = pd.read_csv('C:/Dev/data/prices/366_price_sep2018.csv', usecols = ['authors','title', 'publisher', 'isbn','year','genre'], \
               names = ['authors','title', 'publisher', 'isbn','year','genre'], \
               header=1)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(isbn_check)
# TODO: Process books without ISBN
#noisbn = b[b.isbn == '']
b = b[b.isbn != '']

books_db = books_db.append(b, ignore_index=True, sort=True)

# Read Abris price
b = pd.read_csv('C:/Dev/data/prices/abris_price_oct2017.csv', usecols = ['title_author', 'publisher', 'isbn','year'], \
               names = ['title_author', 'publisher', 'isbn','year'], \
                header=1)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(isbn_check)
# TODO: Process books without ISBN
#noisbn = b[b.isbn == '']
b = b[b.isbn != '']
b[['title','authors']] = b.title_author.str.split('/', 1, expand=True)

books_db = books_db.append(b, ignore_index=True, sort=True)

# Read Abris price (assorted)
b = pd.read_csv('C:/Dev/data/prices/hit_price_oct2018_assort.csv', usecols = ['authors', 'title', 'year', 'isbn', 'publisher'], \
               names = ['series', 'authors', 'title', 'year', 'isbn', 'publisher'], \
                header=1)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(isbn_check)
# TODO: Process books without ISBN
#noisbn = b[b.isbn == '']
b = b[b.isbn != '']

books_db = books_db.append(b, ignore_index=True, sort=True)

# Read Abris price (AST)
b = pd.read_csv('C:/Dev/data/prices/hit_price_oct2018_ast.csv', usecols = ['authors', 'title', 'year', 'isbn', 'publisher'], \
               names = ['authors', 'title', 'year', 'isbn', 'publisher'], \
            header=1)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(isbn_check)
# TODO: Process books without ISBN
#noisbn = b[b.isbn == '']
b = b[b.isbn != '']

books_db = books_db.append(b, ignore_index=True, sort=True)

# Read Infra price
b = pd.read_csv('C:/Dev/data/prices/infra_price_jul2017.csv', usecols = ['title', 'authors', 'publisher', 'year', 'isbn', 'genre1', 'subgenre', 'image'], \
               names = ['c1', 'title', 'authors', 'publisher', 'year', 'isbn', 'genre1', 'subgenre', 'image', 'c2', 'c3'], \
               header=1)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(isbn_check)
# TODO: Process books without ISBN
#noisbn = b[b.isbn == '']
b = b[b.isbn != '']
b['genre'] = b.genre1 + '->' + b.subgenre

books_db = books_db.append(b, ignore_index=True, sort=True)

# Read Labirint price
b = pd.read_csv('C:/Dev/data/prices/labirint_price_apr2018.csv', usecols = ['authors', 'title', 'publisher', 'year', 'isbn'], \
               names = ['authors', 'title', 'publisher', 'series', 'year', 'isbn'], \
                header=1)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(isbn_check)
# TODO: Process books without ISBN
#noisbn = b[b.isbn == '']
b = b[b.isbn != '']

books_db = books_db.append(b, ignore_index=True, sort=True)

# Read URSS price
b = pd.read_csv('C:/Dev/data/prices/urss_price_jul2018.csv', usecols = ['isbn', 'authors', 'title', 'publisher', 'year'], \
               names = ['isbn', 'authors', 'title', 'series', 'publisher', 'year'], \
               header=1)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(isbn_check)
# TODO: Process books without ISBN
#noisbn = b[b.isbn == '']
b = b[b.isbn != '']

books_db = books_db.append(b, ignore_index=True, sort=True)

# Read Amadeus price
b = pd.read_csv('C:/Dev/data/prices/amadeus_05.10.2020.csv', usecols = ['image', 'authors', 'publisher', 'year', 'isbn', 'title', 'genre1', 'genre2', 'subgenre'], \
               names = ['image', 'authors', 'publisher', 'year', 'isbn', 'title', 'genre1', 'genre2', 'subgenre', 'anotation'], \
                header=1)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(isbn_check)
# TODO: Process books without ISBN
#noisbn = b[b.isbn == '']
b = b[b.isbn != '']

b['genre'] = b.genre1 + '->' + b.genre2 + '->' + b.subgenre

books_db = books_db.append(b, ignore_index=True, sort=True)

print('Total number not unique in PRICE:', len(books_db))

books_db = books_db[~books_db.duplicated('isbn')]

print('Total number unique PRICE:', len(books_db))

books_db.to_csv('datasets/prices.csv')

# Merge Biblosphere catalog and price data
b = pd.read_csv('C:/Dev/data/catalog/catalog.csv', usecols = ['isbn', 'authors', 'title', 'lang', 'genre', 'image'], \
               names = ['isbn', 'title', 'authors', 'lexems', 'bookspine', 'lang', 'genre', 'image', 'bid', 'oid', 'description', 'modified'], \
               header=0)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(isbn_check)
# TODO: Process books without ISBN
#noisbn = b[b.isbn == '']
b = b[b.isbn != '']

print('Number of books in catalog:', len(b))

books_db = books_db.append(b, ignore_index=True, sort=True)

print('Total number not unique in ALL:', len(books_db))

books_db = books_db[~books_db.duplicated('isbn')]

print('Total number unique ALL:', len(books_db))

books_db.to_csv('datasets/all.csv')


Total number not unique in PRICE: 306193
Total number unique PRICE: 231486
Number of books in catalog: 375124
Total number not unique in ALL: 606610
Total number unique ALL: 568115


In [47]:
# Merge Biblosphere catalog and price data

b = pd.read_csv('C:/Dev/data/catalog/catalog.csv', usecols = ['isbn', 'authors', 'title', 'lang', 'genre', 'image'], \
               names = ['isbn', 'title', 'authors', 'lexems', 'bookspine', 'lang', 'genre', 'image', 'bid', 'oid', 'description', 'modified'], \
               header=0)

In [ ]:
AlphaFilterId
LanguageId
authorId
editorialId
skip=152
subthemeId
take=12
themeId=8

www.libreriainternacional.com/Libros/GetBooks

In [123]:
import mysql.connector
from mysql.connector import errorcode

TABLES = {}
TABLES['books'] = (
    "CREATE TABLE `books` ("
    "  `bid` int(11) NOT NULL AUTO_INCREMENT,"
    "  `oid` int(11) NOT NULL,"
    "  PRIMARY KEY (`bid`)"
    ")")

TABLES['prints'] = (
    "CREATE TABLE `prints` ("
    "  `isbn` varchar(15) NOT NULL,"
    "  `title` varchar(1024) NOT NULL,"
    "  `authors` varchar(1024) NOT NULL,"
    "  `lexems` varchar(1024) NOT NULL,"
    "  `bookspine` varchar(127) NOT NULL,"
    "  `lang` varchar(127) NOT NULL,"
    "  `genre` varchar(50) NOT NULL,"
    "  `image` varchar(127) NOT NULL,"
    "  `bid` int(11),"
    "  `oid` int(11),"
    "  `description` varchar(1024) NOT NULL,"    
    "  `modified` datetime DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,"    
    "  PRIMARY KEY (`isbn`)"
    ")")

TABLES['tags'] = (
    "CREATE TABLE `tags` ("
    "  `isbn` varchar(15) NOT NULL,"
    "  `tag` varchar(50) NOT NULL,"
    "  PRIMARY KEY (`tag`),"
    "  CONSTRAINT `tags_fk` FOREIGN KEY (`isbn`)"
    "     REFERENCES `prints` (`isbn`) ON DELETE CASCADE"
    ")")

TABLES['bookspines'] = (
    "CREATE TABLE `bookspines` ("
    "  `sid` int(11) NOT NULL AUTO_INCREMENT,"
    "  `bid` int(11) NOT NULL,"
    "  `usid` varchar(31) NOT NULL,"
    "  `imid` varchar(31) NOT NULL,"
    "  `box` varchar(127) NOT NULL,"
    "  `text` varchar(127) NOT NULL,"
    "  PRIMARY KEY (`sid`),"
    "  CONSTRAINT `bookspine_fk` FOREIGN KEY (`bid`)"
    "     REFERENCES `books` (`bid`) ON DELETE CASCADE"
    ")")

TABLES['unknowns'] = (
    "CREATE TABLE `unknowns` ("
    "  `uid` int(11) NOT NULL AUTO_INCREMENT,"
    "  `usid` varchar(31) NOT NULL,"
    "  `imid` varchar(31) NOT NULL,"
    "  `box` varchar(127) NOT NULL,"
    "  `text` varchar(127) NOT NULL,"
    "  `bid` int(11) NOT NULL,"
    "  PRIMARY KEY (`uid`)"
    ")")

cnx = mysql.connector.connect(user='root', password='root',
                              host='35.223.45.184',
                              database='biblosphere',
                              use_pure=False)

cursor = cnx.cursor()

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table books: OK
Creating table prints: OK
Creating table bookspines: OK
Creating table unknowns: OK


In [122]:
cnx = mysql.connector.connect(user='biblosphere', password='biblosphere',
                              host='127.0.0.1',
                              database='biblosphere',
                              use_pure=False)

cursor = cnx.cursor()

cursor.execute("DROP TABLE PRINTS") 
print("Table PRINTS dropped... ")

cursor.execute("DROP TABLE BOOKSPINES") 
print("Table BOOKSPINES dropped... ")

cursor.execute("DROP TABLE BOOKS") 
print("Table BOOKS dropped... ")

cursor.execute("DROP TABLE UNKNOWNS") 
print("Table UNKNOWNS dropped... ")

cursor.close()
cnx.close()



Table PRINTS dropped... 
Table BOOKSPINES dropped... 
Table BOOKS dropped... 
Table UNKNOWNS dropped... 


In [157]:
def lexems(s, full = False):
    if type(s) is str:
        return [w for w in re.sub('[\;\(\)\"\,\/\&\!\?\:\.\-]|#[0-9]*',' ',s.lower()).split() if full or w not in exclude and len(w) > 1]
    elif type(s) is set:
        return [w.lower() for w in s if full or w not in exclude and len(w) > 1]


class Book:
    def __init__(self, isbn, title, authors):
        self.isbn = isbn
        self.title = title
        self.authors = authors
        self.keys = lexems(title + ' ' + authors, full=True)

query = "SELECT isbn, title, authors, MATCH (lexems) AGAINST (? IN BOOLEAN MODE) AS score FROM prints WHERE MATCH (lexems) AGAINST (? IN BOOLEAN MODE) ORDER BY score DESC limit 1"
#query = "SELECT isbn, title, authors, MATCH (lexems) AGAINST ('good to great collins kuku' IN BOOLEAN MODE) AS score FROM prints WHERE MATCH (lexems) AGAINST ('good to great collins kuku' IN BOOLEAN MODE) ORDER BY score DESC limit 1"


def find_book(cursor, words, trace=False):
    # Only match by two words or more
    if len(words) < 1:
        return None, set(), words

    str_words = ' '.join(words)
    cursor.execute(query, (str_words, str_words,))

    results = cursor.fetchall()
    if trace:
        print(results)
    
    
    # Nothing found
    if len(results) == 0:
        return None, set(), words
    
    # Get book information
    isbn, title, authors, score = results[0]
    
    # Get all words from the title and author
    book = Book(isbn, title, authors)
    
    matched_words = set([w for w in words if w in book.keys])
    unmatched_words = words - matched_words

    # Only match by two words or more
    if len(matched_words) < 1:
        return None, set(), words

    if trace:
        print('Book found (%.2f/%d)' % (score, len(matched_words)), title)

    return book, matched_words, unmatched_words
    
    
cnx = mysql.connector.connect(user='biblosphere', password='biblosphere',
                              host='127.0.0.1',
                              database='biblosphere',
                              use_pure=False)

cursor = cnx.cursor(prepared=True)

book, m, u = find_book(cursor, set('rethinking'.split()), trace=False)

if book is not None:
    print(book.authors, book.title, m, u)
else:
    print('Book not found', m, u)

cursor.close()
cnx.close()

Matthew Alan Cahn Rethinking California: Politics and Policy in the Golden State {'rethinking'} {'all'}


In [121]:
import pandas as pd
import numpy as np
import re
import ast

lang_map = {'Русский (Субтитры)': 'rus',
            'ger': 'deu',
            'pol': 'pol',
            'ara': 'ara',
            'Английский': 'eng',
            'Датский': 'dan',
            'Греческий': 'ell',
            'Тибетский': 'bod',
            'Португальский': 'por',
            'tur': 'tur',
            'Иврит': 'heb',
            'vie': 'vie',
            'por': 'por',
            'Китайский': 'zho',
            'Эвенский': 'eve',
            'Турецкий': 'tur',
            'Древнерусский': 'orv',
            'Вьетнамский': 'vie',
            'Пали': 'pli',
            'Молдавский': 'ron',
            'Корякский': 'kpy',
            'Славянский': 'orv',
            'Финский': 'fin',
            'Татарский': 'tat',
            'Словацкий': 'slk',
            'fil': 'fil',
            'mul': 'mul', 
            'Японский': 'jpn', 
            'Баскский': 'eus', 
            'Хорватский': 'hrv', 
            'Литовский': 'lit', 
            'Французский': 'fra' , 
            'Древнегреческий': 'grc', 
            'Нидерландский': 'nld', 
            'Чувашский': 'chv', 
            'nor': 'nor', 
            'Шведский': 'swe', 
            'dan': 'dan', 
            'Испанский': 'spa', 
            'en-US': 'eng', 
            'eng': 'eng', 
            'en-GB': 'eng', 
            'Хинди': 'hin', 
            'Сербский': 'srp', 
            'nl': 'nld', 
            'Старорусский': 'orv', 
            'jpn': 'jpn', 
            'Словенский': 'slv', 
            'Арабский': 'ara', 
            'Болгарский': 'bul', 
            'Венгерский': 'hun', 
            'rum': 'rum', 
            'ita': 'ita', 
            'Таджикский': 'tgk', 
            'Казахский': 'kaz', 
            'Итальянский': 'ita', 
            'Монгольский': 'mgt', 
            'Польский': 'pol', 
            'Санскрит': 'san', 
            'Латинский': 'lat', 
            'fre': 'fra', 
            'swe': 'swe', 
            'Корейский': 'kor', 
            'Древнееврейский': 'hbo', 
            'ind': 'ind', 
            'Голландский': 'nld', 
            'Белорусский': 'bel', 
            'Немецкий': 'deu', 
            'Румынский': 'ron', 
            'Старославянский': 'chu', 
            'Армянский': 'hye', 
            'Чешский': 'ces', 
            'per': 'fas', 
            'Азербайджанский': 'aze', 
            'spa': 'spa', 
            'Русский': 'rus', 
            'rus': 'rus', 
            'Церковнославянский': 'chu', 
            'Эстонский': 'est', 
            'en-CA': 'eng', 
            'Украинский': 'ukr', 
            'Латышский': 'lav', 
            'Арамейский': 'arc',
            'Язык Не Указан': '',
            'Sound Effects Only Track': '',
            '': ''}


def map_lang(s):
    langs = [lang_map[x.strip()] for x in s.split(',')]
    langs = [l for l in langs if l != '']
    langs = ','.join(langs)
    return langs

def parse_list(s):
    try:
        l = ast.literal_eval(s)
        l = list(set([str(i).strip() for i in l]))
        return l
    except ValueError:
        print("Oops! Wrong string", s)
        return ''

def isbn_check(isbn):
    if type(isbn) == float:
        isbn = '{:.0f}'.format(isbn)
    
    if len(isbn) == 10 and re.match(r'^[0-9]*$', isbn) is not None:
        isbn = '978' + isbn
        
    if len(isbn) == 13 and re.match(r'^[0-9]*$', isbn) is not None:
        check = 0
        for i, ch in enumerate(isbn[0:12]):
            check += int(ch) * (1 + i % 2 * 2)

        check = check % 10
        if check > 0:
            check = 10 - check

        return isbn[0:12]+str(check)
    else:
        return ''

    
def build_keys(s):
    s = [w for w in set(re.sub('[\;\(\)\"\,\/\&\!\?\:\.\-\*\·\|\+\$\'\«\@\\\]',' ',s.lower()).split()) if len(w) > 2]
    return ' '.join(s)

books_db = pd.DataFrame(columns=('title', 'authors', 'isbn', 'image', 'lang'))

b = pd.read_csv('../recommender/datasets/books-russian.csv', usecols = ['authors', 'title', 'isbn', 'image', 'lang'], \
               names = ['i','title','authors','isbn','image','price','tags','lang'], \
               header=0)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(parse_list)
b = b.explode('isbn')
b.isbn = b.isbn.apply(isbn_check)

b = b[b.isbn != '']

books_db = books_db.append(b, ignore_index=True, sort=True)

# Read goodbook-10k
b = pd.read_csv('../recommender/datasets/books-goodbooks10k.csv', usecols = ['authors', 'title', 'isbn', 'lang', 'image'], \
              names = ['id','book_id','best_book_id','work_id','books_count','isbn10','isbn','authors','original_publication_year','original_title','title','lang','average_rating','ratings_count','work_ratings_count','work_text_reviews_count','ratings_1','ratings_2','ratings_3','ratings_4','ratings_5','image','small_image'], \
               header=0)

b.fillna(value='', inplace=True)

b.isbn = pd.to_numeric(b.isbn, errors='coerce')
pd.options.display.float_format = '{:.0f}'.format
b.isbn = b.isbn.map('{:.0f}'.format)
b.isbn = b.isbn.apply(isbn_check)
b = b[b.isbn != '']

books_db = books_db.append(b, ignore_index=True, sort=True)

# Read bookcrossing
b = pd.read_csv('../recommender/datasets/books-bookcrossing.csv', usecols = ['authors', 'title', 'isbn', 'image'], \
                names = ['isbn', 'title', 'authors', "Year-Of-Publication", "Publisher", "small_image", "image", "large_image"], \
                sep=';', encoding='latin-1', header=0)

b.fillna(value='', inplace=True)
b.isbn = b.isbn.apply(isbn_check)

b['lang'] = ''
b = b[b.isbn != '']

books_db = books_db.append(b, ignore_index=True, sort=True)

books_db = books_db[~books_db.duplicated('isbn')]

# Map language codes
books_db.lang = books_db.lang.apply(map_lang)

books_db['keys'] = books_db.authors + ' ' + books_db.title
books_db['keys'] = books_db['keys'].apply(build_keys)

books_db.to_csv('datasets/isbn.csv')

books_db.head()

,authors,image,isbn,lang,title,keys
0,,https://cdn1.ozone.ru/multimedia/c250/10239972...,0543209081802,rus,"""-Здравствуйте! Это Я!.."": Стихи",стихи здравствуйте это
1,,https://cdn1.ozone.ru/multimedia/c250/10045153...,9785707000317,,"""... лучше воздержаться""",лучше воздержаться
2,,https://cdn1.ozone.ru/multimedia/c250/10176740...,9785367010756,,"""...Все испытал и все проник""",испытал проник все
3,,https://cdn1.ozone.ru/multimedia/c250/10108160...,9785803201816,rus,"""...Дом твоего детства""",дом детства твоего
4,Чейз Джеймс Хедли,https://cdn1.ozone.ru/multimedia/c250/10047088...,9785726004310,rus,"""...И вы будете редактором отдела"". Двойная сдача",будете сдача редактором отдела чейз хедли джей...


In [120]:
def build_keys(s):
    s = [w for w in set(re.sub('[\;\(\)\"\,\/\&\!\?\:\.\-\*\·\|\+\$\'\«\@\\\]',' ',s.lower()).split()) if len(w) > 2]
    return ' '.join(s)


print(build_keys('Hello\ "world" : hello 1"'))
    

world hello


In [ ]:
# Export table to CSV file
TABLE prints
    INTO OUTFILE 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\prints.txt'
    FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"'; ESCAPED BY '\\'
    LINES TERMINATED BY '\n';
    
# Load data fro mCSV file to table
LOAD DATA INFILE 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\isbn.csv' INTO TABLE prints
    FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"' ESCAPED BY ''
    LINES TERMINATED BY '\r\n'
    IGNORE 1 LINES
    (@dummy, authors, image, isbn, lang, title, lexems)
    SET bookspine = '', genre = '', bid = NULL, oid = NULL;

CREATE FULLTEXT INDEX prints_fulltext ON prints(lexems);

LOAD DATA LOCAL INFILE 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\prints.txt' INTO TABLE prints
    FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"'
    
LOAD DATA LOCAL INFILE 'prints.txt' INTO TABLE prints FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"'

In [ ]:
LOAD DATA LOCAL INFILE 'sql_catalog.csv' INTO TABLE prints
    FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"' ESCAPED BY ''
    LINES TERMINATED BY '\r\n'
    IGNORE 1 LINES
    (isbn, title, authors, lang, genre, image)
    SET bookspine = '', bid = NULL, oid = NULL, description='', lexems = '';

In [5]:
import mysql.connector
from mysql.connector import errorcode

cnx = mysql.connector.connect(user='root', password='root',
                              host='35.223.45.184',
                              database='biblosphere',
                              use_pure=False)

TABLES = {}
TABLES['tags'] = (
    "CREATE TABLE `tags` ("
    "  `tag` varchar(50) NOT NULL,"
    "  `isbn` varchar(15) NOT NULL,"
    "  PRIMARY KEY (`tag`),"
    "  CONSTRAINT `tags_fk` FOREIGN KEY (`isbn`)"
    "     REFERENCES `prints` (`isbn`) ON DELETE CASCADE"
    ")")

cursor = cnx.cursor()

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table tags: OK


In [ ]:
CREATE USER 'biblosphere' IDENTIFIED BY 'biblosphere';
GRANT ALL PRIVILEGES ON biblosphere.* TO 'biblosphere';

In [10]:
import mysql.connector
from mysql.connector import errorcode

cnx = mysql.connector.connect(user='root', password='root',
                              host='35.223.45.184',
                              database='biblosphere',
                              use_pure=False)

cnx.autocommit = True
cursor = cnx.cursor()

# Delete Sliozi predkov
#cursor.execute("DELETE from prints WHERE ISBN='9785911600495'")
#cursor.execute("DELETE from prints WHERE ISBN='9785389015234'")
#cursor.execute("SELECT * from prints WHERE ISBN='9785389015234'")
#cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA='biblosphere' AND TABLE_NAME='prints'")
#cursor.execute("SELECT * from prints limit 1")
#cursor.execute("ALTER TABLE prints ADD COLUMN `modified` datetime DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP")    
cursor.execute("SELECT isbn, title, authors, image, genre, lang, description, MATCH (lexems) AGAINST ('good to great' IN BOOLEAN MODE) AS score FROM prints WHERE MATCH (lexems) AGAINST ('good to great' IN BOOLEAN MODE) ORDER BY score DESC limit 10")

results = cursor.fetchall()
print(results)

cursor.close()
cnx.close()

[('9780062120991', 'Great by Choice: Uncertainty, Chaos, and Luck--Why Some Thrive Despite Them All (Good to Great)', 'Collins, Jim; Hansen, Morten T.', 'https://pictures.abebooks.com/isbn/9780062120991-us-300.jpg', '', '', '', 15.384151458740234), ('9780066620992', "Good to Great: Why Some Companies Make the Leap... and Others Don't", 'James C. Collins', 'https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png', '', 'eng', '', 10.5505952835083), ('9780071388580', "Unbelievably Good Deals &amp; Great Adventures That You Absolutely Can't Get Unless You're Over 50, 2002-2003", 'Joan Rattner Heilman', 'http://images.amazon.com/images/P/0071388583.01.MZZZZZZZ.jpg', '', '', '', 10.5505952835083), ('9780071391146', "Unbelievably Good Deals and Great Adventures That You Absolutely Can't Get Unless You're Over 50, 2003-2004", 'Joan Ratner  Heilman', 'http://images.amazon.com/images/P/0071391142.01.MZZZZZZZ.jpg', '', '', '', 10.5505952835083), ('9780140074635', 

In [2]:
sql_df = pd.read_csv('gs://biblosphere-210106-backups/full-catalog-2020-12-27/Cloud_SQL_Export_2021-01-21 (04:42:22).csv', \
               usecols = ['isbn', 'authors', 'title', 'lang', 'genre', 'image'], \
               names = ['isbn', 'title', 'authors', 'lexems', 'bookspine', 'lang', 'genre', 'image', 'bid', 'oid', 'description', 'modified'], \
               header=0)

print(len(sql_df))
sql_df.to_csv('/data/sql_catalog.csv', index = False)

/home/dstark1977/.local/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


376186


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
genres = {
'Novela contemporánea': "/libros/novela-contemporanea/128000000", 
'Arte': "/libros/arte/101000000",  
'Autoayuda y Espiritualidad': "/libros/autoayuda-y-espiritualidad/102000000", 
'Ciencias Humanas': "/libros/ciencias-humanas/104000000", 
'Ciencias Políticas y Sociales': "/libros/ciencias-politicas-y-sociales/105000000",
'Ciencias': "/libros/ciencias/103000000",
'Cocina': "/libros/cocina/106000000",
'Cómics manga infantil y juvenil': "/libros/comics-manga-infantil-y-juvenil/412000000",
'Cómics': "/libros/comics/411000000", 
'Deportes y juegos': "/libros/deportes-y-juegos/108000000",
'Derecho': "/libros/derecho/109000000",
'Economía': "/libros/economia/110000000", 
'Empresa': "/libros/empresa/111000000", 
'Filología': "/libros/filologia/112000000",
'Fotografía': "/libros/fotografia/113000000",
'Guías de viaje': "/libros/guias-de-viaje/114000000",
'Historia': "/libros/historia/115000000", 
'Idiomas': "/libros/idiomas/116000000", 
'Infantil': "/libros/infantil/117000000",
'Informática': "/libros/informatica/118000000",
'Ingeniería': "/libros/ingenieria/119000000", 
'Juvenil': "/libros/juvenil/117001014", 
'Libros de Texto y Formación': "/libros/libros-de-texto-y-formacion/132000000", 
'Libros latinoamericanos': "/libros/libros-latinoamericanos/417000000", 
'Literatura': "/libros/literatura/121000000", 
'Manualidades': "/libros/manualidades/122000000",
'Medicina': "/libros/medicina/123000000", 
'Música': "/libros/musica/124000000", 
'Narrativa histórica': "/libros/narrativa-historica/125000000", 
'Novela negra': "/libros/novela-negra/126000000", 
'Oposiciones': "/libros/oposiciones/129000000",
'Psicología y Pedagogía': "/libros/psicologia-y-pedagogia/130000000", 
'Romántica y erótica': "/libros/romantica-y-erotica/416000000", 
'Salud y Dietas': "/libros/salud-y-dietas/131000000" 
}

In [4]:
sql_df = pd.read_csv('/data/export.csv', \
               usecols = ['isbn', 'authors', 'title', 'lang', 'genre', 'image'], \
               names = ['isbn', 'title', 'authors', 'lexems', 'bookspine', 'lang', 'genre', 'image', 'bid', 'oid', 'description', 'modified'], \
               header=0)

for g in genres: 
    topic = genres[g]
    topic_name = topic.split('/')[-2]
    b = pd.read_csv('/data/casadellibro_%s.csv' % topic_name, usecols = ['image','authors','title','isbn'])

    sql_df = sql_df.append(b, ignore_index=True)
    
    print('%s: %d records added' % (g, len(b)))

sql_df.fillna(value='', inplace=True)

print('Total records:', len(sql_df))

sql_df = sql_df[~sql_df.duplicated('isbn')]

print('Total unique records:', len(sql_df))

sql_df.to_csv('/data/sql_catalog.csv', index = False)


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Novela contemporánea: 649515 records added
Arte: 74613 records added
Autoayuda y Espiritualidad: 32672 records added
Ciencias Humanas: 152256 records added
Ciencias Políticas y Sociales: 96299 records added
Ciencias: 71362 records added
Cocina: 16563 records added
Cómics manga infantil y juvenil: 9059 records added
Cómics: 33278 records added
Deportes y juegos: 24739 records added
Derecho: 74804 records added
Economía: 36475 records added
Empresa: 42854 records added
Filología: 143056 records added
Fotografía: 11200 records added
Guías de viaje: 34307 records added
Historia: 374529 records added
Idiomas: 59240 records added
Infantil: 156002 records added
Informática: 25352 records added
Ingeniería: 22912 records added
Juvenil: 53774 records added
Libros de Texto y Formación: 103496 records added
Libros latinoamericanos: 6 records added
Literatura: 215750 records added
Manualidades: 5448 records added
Medicina: 34317 records added
Música: 16001 records added
Narrativa histórica: 124683 

In [31]:
# Script to clean data from unprintable characters which fails on loading to MySQL 

sql_df = pd.read_csv('/data/sql_catalog.csv', \
               usecols = ['isbn', 'authors', 'title', 'lang', 'genre', 'image'], \
               header=0)

sql_df.fillna(value='', inplace=True)

sql_df=sql_df[~sql_df.title.str.contains('�')]
sql_df=sql_df[~sql_df.authors.str.contains('�')]
sql_df=sql_df[~sql_df.authors.str.contains('󰥺')]
sql_df=sql_df[~sql_df.authors.str.contains('񡠐')]
sql_df=sql_df[sql_df.title.apply(lambda s: s.isprintable())]
sql_df=sql_df[sql_df.authors.apply(lambda s: s.isprintable())]
sql_df=sql_df[sql_df.authors.apply(lambda s: all([ord(c) < 0xffff for c in s]))]

print(len(sql_df))
sql_df.to_csv('/data/sql_catalog.csv', index = False)

3072531


In [56]:
f=sql_df.authors.str.startswith('ANEL𠡯NGERS')

print(f.sum())

2


In [52]:
'ANEL𠡯NGERS'.isprintable()

True

In [63]:
s = 'ANEL𠡯NGERS'

cond = all([ord(c) < 0xffff for c in s])
print(cond)

True


In [57]:
print(f.index[f].tolist())

[1631697, 1631814]


In [68]:
sql_df=sql_df[sql_df.authors.apply(lambda s: all([ord(c) < 0xffff for c in s]))]
print(len(sql_df))
sql_df.to_csv('/data/sql_catalog.csv', index = False)

3067764


In [66]:
print(sql_df.authors[f])

1631697                              ANEL𠡯NGERS
1631814                              ANEL𠡯NGERS
1647581                              MARVIN M𬬥R
1816929                                PAUL KR𧥲
2211841                JOSEP ANTONI GAR𪐠SIUMELL
2212199                        EDUART TODA Y G𥬬
2212402                        FRAY JOS顤E SIG𥮺A
2212516                         JOS顃ANGA ARG𥬬ES
2213436                    JUAN JOS顓IG𥮺A Y VERA
2213887                   ANTONI AGUST𪐠ALBANELL
2214050                         MAG𡡥RRER I PONS
2214604                       MARIANO CUB𪐠SOLER
2214708                     JULI⮠SEGUNDO DE AG𥲯
2214716                         JOS顃ANGA ARG𥬬ES
2214862                    JUAN JOS顓IG𥮺A Y VERA
2215434                JOSEP ANTONI GAR𪐠SIUMELL
2215437                         JOS顃ANGA ARG𥬬ES
2215891                         JOS顃ANGA ARG𥬬ES
2216149                        FRAY JOS顤E SIG𥮺A
2216470    MEXICO. JUNTA DIRECTIVA DEL DESAG𥠄EL
2216581                       NOEM𤡡POSO 